In [1]:
!unzip "/content/demand-forecasting-duck-product.zip"

Archive:  /content/demand-forecasting-duck-product.zip
  inflating: event_date.parquet      
  inflating: problem2_filter_transaction_price.parquet  
  inflating: problem2_price.parquet  
  inflating: problem2_test.csv       


In [2]:
!pip install uv
!uv pip install -q autogluon.timeseries --system
!uv pip uninstall -q torchaudio torchvision torchtext --system # fix incompatible package versions on Colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 59.0 MB/s eta 0:00:00


In [2]:
import polars as pl
import glob
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import numpy as np
import pandas as pd
import os

In [3]:
all_data = pl.read_parquet("/data/502507_data/duck_stat.parquet")
event = pl.read_parquet("/data/Duck_data/demand-forecasting-duck-product/event_date.parquet")
sub = pl.read_csv('/data/Duck_data/demand-forecasting-duck-product/problem2_test.csv')

In [4]:
all_data

OrderDate,ProductBKey,CustomerBKey,OrderWeight,MODEL_4,mean_weight,median_weight,std_weight,MA_7,SMA_7,EMA_7,STO_14,Momentum_7,max_order_weight,min_order_weight
datetime[ns],str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-01-02 00:00:00,"""000000000023004991""","""2000331295""",12.0,"""ภาคเหนือบน เขต 4""",148.310309,24.0,513.735182,283.485714,283.485714,210.927944,0.0,-312.0,10008.0,6.0
2023-01-02 00:00:00,"""000000000023097050""","""2000112629""",55.2,"""ภาคใต้ เขต 1""",94.879766,39.5,302.866007,279.304286,279.304286,188.425628,0.031236,-29.27,6900.0,12.67
2023-01-02 00:00:00,"""000000000023097050""","""2000006308""",138.0,"""ภาคใต้ เขต 1""",94.879766,39.5,302.866007,241.161429,241.161429,181.278595,0.091106,-267.0,6900.0,12.67
2023-01-02 00:00:00,"""000000000023004991""","""2000265032""",12.0,"""ภาคใต้ เขต 1""",148.310309,24.0,513.735182,43.59,43.59,157.680926,0.0,-1383.0,10008.0,6.0
2023-01-02 00:00:00,"""000000000023097053""","""2000169375""",60.0,"""ภาคใต้ เขต 1""",73.815338,30.0,193.807638,50.447143,50.447143,144.257388,0.034707,48.0,6020.0,8.75
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-01-18 00:00:00,"""000000000023097377""","""2000335629""",8.0,"""ภาคใต้ เขต 4""",26.240994,16.0,57.972189,23.371429,23.371429,24.321085,0.050847,-22.0,1200.0,2.0
2025-01-18 00:00:00,"""000000000023097377""","""2000167574""",24.0,"""ภาคใต้ เขต 4""",26.240994,16.0,57.972189,22.514286,22.514286,24.280949,0.186441,-6.0,1200.0,2.0
2025-01-18 00:00:00,"""000000000023097377""","""2000222813""",48.0,"""ภาคใต้ เขต 4""",26.240994,16.0,57.972189,24.742857,24.742857,27.245831,0.389831,15.6,1200.0,2.0


In [16]:
all_data = all_data.drop_nulls()

In [17]:
all_data

OrderDate,CustomerBKey,ProductBKey,MODEL_4,OrderWeight,mean_weight,median_weight,std_weight,MA_7,SMA_7,EMA_7,STO_14,Momentum_7,max_order_weight,min_order_weight
datetime[μs],str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-06-04 00:00:00,"""2000389390""","""000000000023038994""","""ภาคกลางตะวันตก เขต 2""",2.25,14.315157,4.5,39.105999,181.75,181.75,158.720479,0.0,-17.75,399.75,0.15
2024-09-07 00:00:00,"""2000335288""","""000000000023057674""","""ภาคกลางตะวันตก เขต 2""",10.0,16.296582,10.0,28.385846,27.714286,27.714286,36.268902,0.558659,5.0,4000.0,1.0
2024-03-27 00:00:00,"""2000019975""","""000000000023001078""","""ภาคกลางตะวันตก เขต 2""",20.0,33.631441,15.0,201.224689,17.857143,17.857143,72.461587,1.069519,-1480.0,11420.0,5.0
2024-05-21 00:00:00,"""2000088230""","""000000000023028201""","""ภาคกลางตะวันตก เขต 2""",5.0,22.786017,10.0,62.316487,21.928571,21.928571,46.993529,0.125104,-1.0,500.0,1.0
2024-11-07 00:00:00,"""2000019975""","""000000000023001659""","""ภาคกลางตะวันตก เขต 2""",10.0,23.011945,10.0,36.606269,9.614286,9.614286,16.226785,2.749832,0.0,5050.0,5.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-07-26 00:00:00,"""2000370395""","""000000000023015376""","""ภาคเหนือบน เขต 4""",300.0,52.340395,20.0,159.608387,114.142857,114.142857,119.509854,100.0,290.0,5000.0,3.03
2023-01-22 00:00:00,"""2000333318""","""000000000023068096""","""ภาคเหนือบน เขต 4""",120.0,228.034668,120.0,818.769262,61.428571,61.428571,77.338789,38.983051,-90.0,36000.0,80.0
2023-08-09 00:00:00,"""2000340565""","""000000000023092320""","""ภาคเหนือบน เขต 4""",9.0,26.100378,10.0,113.175572,36.142857,36.142857,588.482838,0.016669,1.3,1010.0,5.0


**Change date format**

In [5]:
all_data = all_data.with_columns(pl.col("OrderDate").cast(pl.Date).alias("OrderDate"))
event = event.with_columns(pl.col("date").cast(pl.Date).alias("date"))
sub = sub.with_columns(pl.col("OrderDate").cast(pl.Date).alias("OrderDate"))

**Create key_cv_p in submiss file**

In [6]:
sub_ = sub.with_columns((pl.col("CustomerBKey") + "+0000000000" + pl.col("ProductBKey").cast(pl.Utf8)).alias("key_cv_p"))
sub_

id,OrderDate,CustomerBKey,ProductBKey,horizon,MODEL_4,ProductForPlan1,ProductForPlan10,OrderWeight,key_cv_p
str,date,str,i64,i64,str,str,str,f64,str
"""f3cd25af-da90-427d-97bb-b84680…",2025-01-19,"""020P""",23013118,1,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",0.0,"""020P+000000000023013118"""
"""059cc414-0587-4f07-bb58-3aa2c8…",2025-01-20,"""020P""",23013118,2,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",0.0,"""020P+000000000023013118"""
"""c1de6281-3fc1-495a-aac9-45d818…",2025-01-21,"""020P""",23013118,3,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",0.0,"""020P+000000000023013118"""
"""3a8267d7-8fc5-4a24-8005-e206f4…",2025-01-22,"""020P""",23013118,4,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",null,"""020P+000000000023013118"""
"""7eea9f7a-da37-4b1c-a183-696d80…",2025-01-23,"""020P""",23013118,5,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",null,"""020P+000000000023013118"""
…,…,…,…,…,…,…,…,…,…
"""d8626342-58ed-423c-870d-16c41e…",2025-02-04,"""NE37""",23105810,17,"""ภาคอีสานล่าง เขต 2""","""Duck""","""หลอดอาหารเป็ด""",null,"""NE37+000000000023105810"""
"""9894c71a-4a66-43d9-9512-954b0b…",2025-02-05,"""NE37""",23105810,18,"""ภาคอีสานล่าง เขต 2""","""Duck""","""หลอดอาหารเป็ด""",null,"""NE37+000000000023105810"""
"""3fe03eca-a826-4101-ab69-db9b65…",2025-02-06,"""NE37""",23105810,19,"""ภาคอีสานล่าง เขต 2""","""Duck""","""หลอดอาหารเป็ด""",null,"""NE37+000000000023105810"""


**select feature for train**

In [30]:
sorted = all_data.sort("OrderDate")
sorted

key_cv_p,OrderDate,OrderWeight,mean_weight,median_weight,std_weight,MA_7,SMA_7,EMA_7,STO_14,Momentum_7,max_order_weight,min_order_weight
str,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2000090793+000000000023016259""",2023-01-01,1500.0,891.911418,600.0,1393.075236,303.285714,303.285714,448.779418,100.0,1448.44,90900.0,30.0
"""2000254247+000000000023068097""",2023-01-01,210.0,228.034668,120.0,818.769262,325.0,325.0,319.000922,13.88518,197.0,15750.0,50.0
"""2000113812+000000000023009027""",2023-01-01,10.0,20.825511,10.0,84.045511,13.571429,13.571429,14.177203,23.076923,-5.0,1890.0,5.0
"""2000357596+000000000023068099""",2023-01-01,105.0,228.034668,120.0,818.769262,37.857143,37.857143,75.690875,8.368201,100.0,4201.0,72.4
"""2000237511+000000000023000188""",2023-01-01,5.0,8.845986,6.0,20.857429,58.0,58.0,42.400711,0.0,-15.0,1700.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…
"""2000017422+000000000023057696""",2025-04-03,20.0,46.579389,20.0,118.124839,15.0,15.0,17.636193,34.782609,16.0,4000.0,1.0
"""2000260624+000000000023057673""",2025-04-03,9.0,23.913178,12.0,44.222691,167.0,167.0,196.513895,0.446927,-1.0,3000.0,1.0
"""2000149357+000000000023001605""",2025-04-03,40.0,398.826289,50.0,1080.791794,195.714286,195.714286,219.990217,4.017857,36.0,15700.0,10.0


In [7]:
all_data = all_data.with_columns((pl.col("CustomerBKey") + "+" + pl.col("ProductBKey").cast(pl.Utf8)).alias("key_cv_p"))
all_data

OrderDate,ProductBKey,CustomerBKey,OrderWeight,MODEL_4,mean_weight,median_weight,std_weight,MA_7,SMA_7,EMA_7,STO_14,Momentum_7,max_order_weight,min_order_weight,key_cv_p
date,str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
2023-01-02,"""000000000023004991""","""2000331295""",12.0,"""ภาคเหนือบน เขต 4""",148.310309,24.0,513.735182,283.485714,283.485714,210.927944,0.0,-312.0,10008.0,6.0,"""2000331295+000000000023004991"""
2023-01-02,"""000000000023097050""","""2000112629""",55.2,"""ภาคใต้ เขต 1""",94.879766,39.5,302.866007,279.304286,279.304286,188.425628,0.031236,-29.27,6900.0,12.67,"""2000112629+000000000023097050"""
2023-01-02,"""000000000023097050""","""2000006308""",138.0,"""ภาคใต้ เขต 1""",94.879766,39.5,302.866007,241.161429,241.161429,181.278595,0.091106,-267.0,6900.0,12.67,"""2000006308+000000000023097050"""
2023-01-02,"""000000000023004991""","""2000265032""",12.0,"""ภาคใต้ เขต 1""",148.310309,24.0,513.735182,43.59,43.59,157.680926,0.0,-1383.0,10008.0,6.0,"""2000265032+000000000023004991"""
2023-01-02,"""000000000023097053""","""2000169375""",60.0,"""ภาคใต้ เขต 1""",73.815338,30.0,193.807638,50.447143,50.447143,144.257388,0.034707,48.0,6020.0,8.75,"""2000169375+000000000023097053"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-01-18,"""000000000023097377""","""2000335629""",8.0,"""ภาคใต้ เขต 4""",26.240994,16.0,57.972189,23.371429,23.371429,24.321085,0.050847,-22.0,1200.0,2.0,"""2000335629+000000000023097377"""
2025-01-18,"""000000000023097377""","""2000167574""",24.0,"""ภาคใต้ เขต 4""",26.240994,16.0,57.972189,22.514286,22.514286,24.280949,0.186441,-6.0,1200.0,2.0,"""2000167574+000000000023097377"""
2025-01-18,"""000000000023097377""","""2000222813""",48.0,"""ภาคใต้ เขต 4""",26.240994,16.0,57.972189,24.742857,24.742857,27.245831,0.389831,15.6,1200.0,2.0,"""2000222813+000000000023097377"""


In [23]:
all_data

OrderDate,CustomerBKey,ProductBKey,MODEL_4,OrderWeight,mean_weight,median_weight,std_weight,MA_7,SMA_7,EMA_7,STO_14,Momentum_7,max_order_weight,min_order_weight,key_cv_p
date,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
2024-06-04,"""2000389390""","""000000000023038994""","""ภาคกลางตะวันตก เขต 2""",2.25,14.315157,4.5,39.105999,181.75,181.75,158.720479,0.0,-17.75,399.75,0.15,"""2000389390+000000000023038994"""
2024-09-07,"""2000335288""","""000000000023057674""","""ภาคกลางตะวันตก เขต 2""",10.0,16.296582,10.0,28.385846,27.714286,27.714286,36.268902,0.558659,5.0,4000.0,1.0,"""2000335288+000000000023057674"""
2024-03-27,"""2000019975""","""000000000023001078""","""ภาคกลางตะวันตก เขต 2""",20.0,33.631441,15.0,201.224689,17.857143,17.857143,72.461587,1.069519,-1480.0,11420.0,5.0,"""2000019975+000000000023001078"""
2024-05-21,"""2000088230""","""000000000023028201""","""ภาคกลางตะวันตก เขต 2""",5.0,22.786017,10.0,62.316487,21.928571,21.928571,46.993529,0.125104,-1.0,500.0,1.0,"""2000088230+000000000023028201"""
2024-11-07,"""2000019975""","""000000000023001659""","""ภาคกลางตะวันตก เขต 2""",10.0,23.011945,10.0,36.606269,9.614286,9.614286,16.226785,2.749832,0.0,5050.0,5.0,"""2000019975+000000000023001659"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-07-26,"""2000370395""","""000000000023015376""","""ภาคเหนือบน เขต 4""",300.0,52.340395,20.0,159.608387,114.142857,114.142857,119.509854,100.0,290.0,5000.0,3.03,"""2000370395+000000000023015376"""
2023-01-22,"""2000333318""","""000000000023068096""","""ภาคเหนือบน เขต 4""",120.0,228.034668,120.0,818.769262,61.428571,61.428571,77.338789,38.983051,-90.0,36000.0,80.0,"""2000333318+000000000023068096"""
2023-08-09,"""2000340565""","""000000000023092320""","""ภาคเหนือบน เขต 4""",9.0,26.100378,10.0,113.175572,36.142857,36.142857,588.482838,0.016669,1.3,1010.0,5.0,"""2000340565+000000000023092320"""


In [9]:
all_data = all_data.select(["key_cv_p", "OrderDate", "OrderWeight", "mean_weight", "median_weight", "EMA_7", "Momentum_7"])
all_data

key_cv_p,OrderDate,OrderWeight,mean_weight,median_weight,EMA_7,Momentum_7
str,date,f64,f64,f64,f64,f64
"""2000331295+000000000023004991""",2023-01-02,12.0,148.310309,24.0,210.927944,-312.0
"""2000112629+000000000023097050""",2023-01-02,55.2,94.879766,39.5,188.425628,-29.27
"""2000006308+000000000023097050""",2023-01-02,138.0,94.879766,39.5,181.278595,-267.0
"""2000265032+000000000023004991""",2023-01-02,12.0,148.310309,24.0,157.680926,-1383.0
"""2000169375+000000000023097053""",2023-01-02,60.0,73.815338,30.0,144.257388,48.0
…,…,…,…,…,…,…
"""2000335629+000000000023097377""",2025-01-18,8.0,26.240994,16.0,24.321085,-22.0
"""2000167574+000000000023097377""",2025-01-18,24.0,26.240994,16.0,24.280949,-6.0
"""2000222813+000000000023097377""",2025-01-18,48.0,26.240994,16.0,27.245831,15.6


**Delete key_cv_p not in submiss file**

In [8]:
sub_

id,OrderDate,CustomerBKey,ProductBKey,horizon,MODEL_4,ProductForPlan1,ProductForPlan10,OrderWeight,key_cv_p
str,date,str,i64,i64,str,str,str,f64,str
"""f3cd25af-da90-427d-97bb-b84680…",2025-01-19,"""020P""",23013118,1,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",0.0,"""020P+000000000023013118"""
"""059cc414-0587-4f07-bb58-3aa2c8…",2025-01-20,"""020P""",23013118,2,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",0.0,"""020P+000000000023013118"""
"""c1de6281-3fc1-495a-aac9-45d818…",2025-01-21,"""020P""",23013118,3,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",0.0,"""020P+000000000023013118"""
"""3a8267d7-8fc5-4a24-8005-e206f4…",2025-01-22,"""020P""",23013118,4,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",null,"""020P+000000000023013118"""
"""7eea9f7a-da37-4b1c-a183-696d80…",2025-01-23,"""020P""",23013118,5,"""กรุงเทพฯ เขต4""","""Duck""","""เป็ดพะโล้""",null,"""020P+000000000023013118"""
…,…,…,…,…,…,…,…,…,…
"""d8626342-58ed-423c-870d-16c41e…",2025-02-04,"""NE37""",23105810,17,"""ภาคอีสานล่าง เขต 2""","""Duck""","""หลอดอาหารเป็ด""",null,"""NE37+000000000023105810"""
"""9894c71a-4a66-43d9-9512-954b0b…",2025-02-05,"""NE37""",23105810,18,"""ภาคอีสานล่าง เขต 2""","""Duck""","""หลอดอาหารเป็ด""",null,"""NE37+000000000023105810"""
"""3fe03eca-a826-4101-ab69-db9b65…",2025-02-06,"""NE37""",23105810,19,"""ภาคอีสานล่าง เขต 2""","""Duck""","""หลอดอาหารเป็ด""",null,"""NE37+000000000023105810"""


In [10]:
z = sub_.to_pandas()
# Get unique item_ids from the Polars DataFrame
unique_item_ids = z['key_cv_p'].unique()

# Filter the Polars DataFrame with Polars syntax
filtered_ts_df = all_data.filter(pl.col('key_cv_p').is_in(unique_item_ids))

# Check the shape before and after filtering
print(f"Original shape: {all_data.shape}")
print(f"Filtered shape: {filtered_ts_df.shape}")

# Verify that all item_ids in the filtered dataframe are in the unique list
remaining_item_ids = filtered_ts_df['key_cv_p'].unique()
print(f"Number of unique item_ids in filtered data: {len(remaining_item_ids)}")

Original shape: (540551, 7)
Filtered shape: (382655, 7)
Number of unique item_ids in filtered data: 10327


**Group by OrderDate and key_cv_p**

In [11]:
# Now group by date and item_id to handle duplicates
consolidated_data = (
    filtered_ts_df
    .group_by(['OrderDate', 'key_cv_p', "mean_weight", "median_weight", "EMA_7", "Momentum_7"])
    .agg([
        pl.col('OrderWeight').median().alias('OrderWeight_median'),  # Median sales value
        pl.col('OrderWeight').mean().alias('OrderWeight_mean'), # Sum of units sold
    ])
    .sort(['OrderDate', 'key_cv_p'])
)
consolidated_data

OrderDate,key_cv_p,mean_weight,median_weight,EMA_7,Momentum_7,OrderWeight_median,OrderWeight_mean
date,str,f64,f64,f64,f64,f64,f64
2023-01-02,"""2000006308+000000000023097050""",94.879766,39.5,181.278595,-267.0,138.0,138.0
2023-01-02,"""2000024215+000000000023097056""",81.006346,32.4,201.546881,-26.2,29.0,29.0
2023-01-02,"""2000024215+000000000023097059""",97.537742,35.0,162.532823,36.0,48.0,48.0
2023-01-02,"""2000112629+000000000023097050""",94.879766,39.5,188.425628,-29.27,55.2,55.2
2023-01-02,"""2000146338+000000000023097056""",81.006346,32.4,106.526418,58.0,87.0,87.0
…,…,…,…,…,…,…,…
2025-01-18,"""EA27+000000000023103880""",73.766732,30.0,108.261233,6.0,30.0,30.0
2025-01-18,"""EA27+000000000023103882""",72.296912,40.0,98.778578,12.4,32.4,32.4
2025-01-18,"""N024+000000000023084665""",12.373069,9.84,33.584573,47.2,49.2,49.2


**Change data to TimeseriesDataframe (Autoguon format)**

In [12]:
df_transformed = (
    consolidated_data
    .rename({
        "OrderDate": "timestamp",
        "key_cv_p": "item_id",
        "OrderWeight_median": "target"

    })
    .with_columns(
        pl.col("target").fill_null(0)  # Fill nulls with appropriate values
    )
    .sort(["item_id", "timestamp"])
)

df_transformed

timestamp,item_id,mean_weight,median_weight,EMA_7,Momentum_7,target,OrderWeight_mean
date,str,f64,f64,f64,f64,f64,f64
2023-01-17,"""020P+000000000023013118""",41.871714,14.14,43.701702,-11.72,28.28,28.28
2023-01-19,"""020P+000000000023013118""",41.871714,14.14,52.880698,85.24,125.24,125.24
2023-01-20,"""020P+000000000023013118""",41.871714,14.14,154.663414,618.2,626.2,626.2
2023-01-21,"""020P+000000000023013118""",41.871714,14.14,70.267756,-407.98,2.02,2.02
2023-01-23,"""020P+000000000023013118""",41.871714,14.14,47.441627,-22.92,8.08,8.08
…,…,…,…,…,…,…,…
2024-02-28,"""NE37+000000000023097377""",26.240994,16.0,17.433907,-52.0,8.0,8.0
2024-08-14,"""NE37+000000000023097377""",26.240994,16.0,38.677951,18.5,32.0,32.0
2024-10-11,"""NE37+000000000023097377""",26.240994,16.0,28.85279,-16.0,8.0,8.0


**Check range of date each item_id**

In [13]:
date_ranges = (
    df_transformed
    .group_by('item_id')
    .agg([
        pl.col('timestamp').min().alias('earliest_date'),
        pl.col('timestamp').max().alias('latest_date')
    ])
)
date_ranges

item_id,earliest_date,latest_date
str,date,date
"""020P+000000000023013118""",2023-01-17,2025-01-10
"""020P+000000000023031706""",2023-02-04,2025-01-04
"""020P+000000000023041352""",2024-07-02,2025-01-16
"""020P+000000000023048743""",2023-04-08,2025-01-09
"""020P+000000000023048752""",2024-05-23,2024-11-06
…,…,…
"""NE37+000000000023094972""",2023-02-22,2025-01-15
"""NE37+000000000023097056""",2024-02-02,2025-01-15
"""NE37+000000000023097058""",2023-01-11,2024-12-23


**Fill 0 in range 2023-01-01 to 2025-01-18**

In [14]:
import polars as pl
from datetime import date

# Step 1: Create a complete date range using a different approach
start_date = date(2023, 1, 1)
end_date = date(2025, 1, 18)
dates_df = pl.DataFrame({
    "timestamp": pl.date_range(start_date, end_date, "1d", eager=True)
})

# Step 2: Get unique item_ids
unique_item_ids = df_transformed.get_column("item_id").unique()

# Step 3: Create DataFrame for the items
items_df = pl.DataFrame({"item_id": unique_item_ids})

# Create a cross product of dates and item_ids
complete_df = dates_df.join(items_df, how="cross")

# Step 4: Merge with original data to bring in existing values
result = complete_df.join(
    df_transformed,
    on=["item_id", "timestamp"],
    how="left"
)

# Step 5: Fill missing target values with 0
result = result.with_columns(
    pl.col("target").fill_null(0),
    pl.col("OrderWeight_mean").fill_null(0)
)

# Sort by item_id and timestamp
result = result.sort(["item_id", "timestamp"])

result

timestamp,item_id,mean_weight,median_weight,EMA_7,Momentum_7,target,OrderWeight_mean
date,str,f64,f64,f64,f64,f64,f64
2023-01-01,"""020P+000000000023013118""",null,null,null,null,0.0,0.0
2023-01-02,"""020P+000000000023013118""",null,null,null,null,0.0,0.0
2023-01-03,"""020P+000000000023013118""",null,null,null,null,0.0,0.0
2023-01-04,"""020P+000000000023013118""",null,null,null,null,0.0,0.0
2023-01-05,"""020P+000000000023013118""",null,null,null,null,0.0,0.0
…,…,…,…,…,…,…,…
2025-01-14,"""NE37+000000000023097377""",null,null,null,null,0.0,0.0
2025-01-15,"""NE37+000000000023097377""",null,null,null,null,0.0,0.0
2025-01-16,"""NE37+000000000023097377""",null,null,null,null,0.0,0.0


## Feature engineer

add holiday

In [15]:
event_columns = [
    "is_new_year", "is_halloween", "is_songkarn", "is_loykrathong",
    "is_thudjene", "is_sarthjene", "is_j_dates", "is_holiday",
    "is_makabusha", "is_visakabucha", "is_asalha", "is_khao_pansa", "is_ok_pansa"
]

df = event.with_columns(
    (pl.sum_horizontal(event_columns) > 0).cast(pl.Int8).alias("event_flag")
)

holiday = df.select(["date", "event_flag"])
holiday = holiday.rename({"date":"timestamp"})
holiday

timestamp,event_flag
date,i8
2023-01-01,1
2023-01-02,0
2023-01-03,0
2023-01-04,0
2023-01-05,0
…,…
2025-04-26,0
2025-04-27,0
2025-04-28,0


In [16]:
result_ = result.join(
    holiday,
    left_on="timestamp",
    right_on="timestamp",
    how="left"
)
result_

timestamp,item_id,mean_weight,median_weight,EMA_7,Momentum_7,target,OrderWeight_mean,event_flag
date,str,f64,f64,f64,f64,f64,f64,i8
2023-01-01,"""020P+000000000023013118""",null,null,null,null,0.0,0.0,1
2023-01-02,"""020P+000000000023013118""",null,null,null,null,0.0,0.0,0
2023-01-03,"""020P+000000000023013118""",null,null,null,null,0.0,0.0,0
2023-01-04,"""020P+000000000023013118""",null,null,null,null,0.0,0.0,0
2023-01-05,"""020P+000000000023013118""",null,null,null,null,0.0,0.0,0
…,…,…,…,…,…,…,…,…
2025-01-14,"""NE37+000000000023097377""",null,null,null,null,0.0,0.0,0
2025-01-15,"""NE37+000000000023097377""",null,null,null,null,0.0,0.0,0
2025-01-16,"""NE37+000000000023097377""",null,null,null,null,0.0,0.0,0


set index

In [17]:
pandas_df = result_.to_pandas()

pandas_df = pandas_df.set_index(['item_id', 'timestamp'])
pandas_df

mean_weight  median_weight  EMA_7  \
item_id                 timestamp                                       
020P+000000000023013118 2023-01-01          NaN            NaN    NaN   
                        2023-01-02          NaN            NaN    NaN   
                        2023-01-03          NaN            NaN    NaN   
                        2023-01-04          NaN            NaN    NaN   
                        2023-01-05          NaN            NaN    NaN   
...                                         ...            ...    ...   
NE37+000000000023097377 2025-01-14          NaN            NaN    NaN   
                        2025-01-15          NaN            NaN    NaN   
                        2025-01-16          NaN            NaN    NaN   
                        2025-01-17          NaN            NaN    NaN   
                        2025-01-18          NaN            NaN    NaN   

                                    Momentum_7  target  OrderWeight_mean  \
item_id                 timestamp                                          
020P+000000000023013118 2023-01-01         NaN     0.0               0.0   
                        2023-01-02         NaN     0.0               0.0   
                        2023-01-03         NaN     0.0               0.0   
                        2023-01-04         NaN     0.0               0.0   
                        2023-01-05         NaN     0.0               0.0   
...                                        ...     ...               ...   
NE37+000000000023097377 2025-01-14         NaN     0.0               0.0   
                        2025-01-15         NaN     0.0               0.0   
                        2025-01-16         NaN     0.0               0.0   
                        2025-01-17         NaN     0.0               0.0   
                        2025-01-18         NaN     0.0               0.0   

                                    event_flag  
item_id                 timestamp               
020P+000000000023013118 2023-01-01           1  
                        2023-01-02           0  
                        2023-01-03           0  
                        2023-01-04           0  
                        2023-01-05           0  
...                                        ...  
NE37+000000000023097377 2025-01-14           0  
                        2025-01-15           0  
                        2025-01-16           0  
                        2025-01-17           0  
                        2025-01-18           0  

[7736138 rows x 7 columns]

**convert to format TimeSeriesDataFrame**

In [18]:
# Correct method for TimeSeriesDataFrame creation
ts_df = TimeSeriesDataFrame.from_data_frame(
    pandas_df,
    id_column="item_id",
    timestamp_column="timestamp",
)
ts_df

mean_weight  median_weight  EMA_7  \
item_id                 timestamp                                       
020P+000000000023013118 2023-01-01          NaN            NaN    NaN   
                        2023-01-02          NaN            NaN    NaN   
                        2023-01-03          NaN            NaN    NaN   
                        2023-01-04          NaN            NaN    NaN   
                        2023-01-05          NaN            NaN    NaN   
...                                         ...            ...    ...   
NE37+000000000023097377 2025-01-14          NaN            NaN    NaN   
                        2025-01-15          NaN            NaN    NaN   
                        2025-01-16          NaN            NaN    NaN   
                        2025-01-17          NaN            NaN    NaN   
                        2025-01-18          NaN            NaN    NaN   

                                    Momentum_7  target  OrderWeight_mean  \
item_id                 timestamp                                          
020P+000000000023013118 2023-01-01         NaN     0.0               0.0   
                        2023-01-02         NaN     0.0               0.0   
                        2023-01-03         NaN     0.0               0.0   
                        2023-01-04         NaN     0.0               0.0   
                        2023-01-05         NaN     0.0               0.0   
...                                        ...     ...               ...   
NE37+000000000023097377 2025-01-14         NaN     0.0               0.0   
                        2025-01-15         NaN     0.0               0.0   
                        2025-01-16         NaN     0.0               0.0   
                        2025-01-17         NaN     0.0               0.0   
                        2025-01-18         NaN     0.0               0.0   

                                    event_flag  
item_id                 timestamp               
020P+000000000023013118 2023-01-01           1  
                        2023-01-02           0  
                        2023-01-03           0  
                        2023-01-04           0  
                        2023-01-05           0  
...                                        ...  
NE37+000000000023097377 2025-01-14           0  
                        2025-01-15           0  
                        2025-01-16           0  
                        2025-01-17           0  
                        2025-01-18           0  

[7736138 rows x 7 columns]

## Train model

In [19]:
# Now create the predictor with the same frequency
prediction_length = 21 # days to forcasting
predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    freq="D"
)

# Split the data
train_data, test_data = ts_df.train_test_split(prediction_length)


In [21]:
predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    freq="D",  # Specify daily frequency
    target="target",
    known_covariates_names=["OrderWeight_mean", "mean_weight"],
    eval_metric='mae'
).fit(
    train_data,
    presets="medium_quality",
    hyperparameters={
        "DirectTabular": [
            {
            },
    #         # {
    #         #     "model_path": "bolt_large",
    #         #     "covariate_regressor": "CAT",
    #         #     "target_scaler": "standard",
    #         #     "ag_args": {"name_suffix": "WithRegressor"},
    #         # },
        ],
    },
    enable_ensemble=False,
    time_limit=3600,
    random_seed = 3
)

Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to '/data/TT/AutogluonModels/ag-20250521_164243'
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.10.17
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #149-Ubuntu SMP Fri Apr 11 22:06:13 UTC 2025
CPU Count:          12
GPU Count:          1
Memory Avail:       41.16 GB / 196.57 GB (20.9%)
Disk Space Avail:   223.61 GB / 471.40 GB (47.4%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': False,
 'eval_metric': MAE,
 'freq': 'D',
 'hyperparameters': {'DirectTabular': [{}]},
 'known_covariates_names': ['OrderWeight_mean', 'mean_weight'],
 'num_val_windows': 1,
 'prediction_length': 21,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 3,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 3600,
 'verbosity': 

In [63]:
predictor.feature_importance(test_data)

Computing feature importance
/home/siamai/anaconda3/envs/torch-gpu/lib/python3.10/site-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated and will be removed in v2.0. Please set the `eval_metric.prediction_length` attribute instead.
  warnings.warn(
/home/siamai/anaconda3/envs/torch-gpu/lib/python3.10/site-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated and will be removed in v2.0. Please set the `eval_metric.prediction_length` attribute instead.
  warnings.warn(
/home/siamai/anaconda3/envs/torch-gpu/lib/python3.10/site-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated and will be removed in v2.0. Please set the `eval_metric.prediction_length` attribute instead.
  warnings.warn(
/home/siamai/anaconda3/envs/torc

,importance,stdev,n,p99_low,p99_high
OrderWeight_mean,2.236321,0.757184,5.0,0.677270,3.795373
event_flag,-0.001832,0.006967,5.0,-0.016176,0.012512
IsHoliday,-0.021536,0.013495,5.0,-0.049322,0.006249
customer_order_frequency,0.000000,0.000000,5.0,0.000000,0.000000
IsWeekend,-0.004823,0.016195,5.0,-0.038169,0.028522


In [21]:
predictor.leaderboard(test_data)


Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).
/usr/local/lib/python3.11/dist-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated and will be removed in v2.0. Please set the `eval_metric.prediction_length` attribute instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated and will be removed in v2.0. Please set the `eval_metric.prediction_length` attribute instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated and will be removed in v2.0. Please set the `eval_metric.prediction_length` attribute instead.
  warnings.warn(
/usr/local/

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,DirectTabular,-1.425374,-1.494522,16.553089,16.716437,64.989551,3
1,RecursiveTabular,-3.151293,-2.911134,96.546284,83.051186,439.259231,2
2,DeepAR,-3.543114,-3.859622,490.618896,953.434804,292.059180,5
3,TemporalFusionTransformer,-4.803113,-4.763252,27.159464,26.165508,299.275849,4
4,SeasonalNaive,-5.841367,-4.489021,29.598266,31.103610,13.834151,1


### preprocess test data

add feature mean

In [22]:
ss = pl.read_csv('/kaggle/input/median-duckkk/duck_median.csv')
feature_mean = ss.select(['id', 'OrderWeight'])
feature_mean= sub_.join(
    ss,
    left_on="id",
    right_on="id",
    how="inner"
)
feature_mean

FileNotFoundError: No such file or directory (os error 2): /kaggle/input/median-duckkk/duck_median.csv

In [ ]:
# เลือกคอลัมน์ให้เหมือน train
test_df = feature_mean.select(['OrderDate','key_cv_p','OrderWeight_right'])
# แปลง format str to date
test_df = test_df.with_columns(
    pl.col("OrderDate").cast(pl.Date).alias("OrderDate")
)
test_df

**rename column**

In [ ]:
test_df1 = (
    test_df
    .rename({
        "OrderDate": "timestamp",
        "key_cv_p": "item_id",
        "OrderWeight_right": "OrderWeight_mean"

    })
    .sort(["item_id", "timestamp"])
)

test_df1

**Delete key_cv_p not in train**
key_cv_p ไม่มีใน train จะ predict ไม่ได้

In [ ]:
test_df1 = test_df1.to_pandas()
result__ = result_.to_pandas()

test_df1 = test_df1[test_df1['item_id'].isin(result__['item_id'].unique())]
test_df1 = pl.from_pandas(test_df1)
test_df1

In [ ]:
# แปลง format str to date
test_df1 = test_df1.with_columns(
    pl.col("timestamp").cast(pl.Date).alias("timestamp"))
test_df1

In [23]:
predictor.feature_importance(train_data)

Computing feature importance
/usr/local/lib/python3.11/dist-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated and will be removed in v2.0. Please set the `eval_metric.prediction_length` attribute instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated and will be removed in v2.0. Please set the `eval_metric.prediction_length` attribute instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `prediction_length` to `TimeSeriesScorer.__call__` is deprecated and will be removed in v2.0. Please set the `eval_metric.prediction_length` attribute instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/autogluon/timeseries/metrics/abstract.py:101: FutureWarning: Passing `pre

,importance,stdev,n,p99_low,p99_high
OrderWeight_mean,2.767408,1.388150,5.0,-0.090812,5.625627
event_flag,0.000114,0.000149,5.0,-0.000193,0.000422


Check range of date each item_id **like trian**

In [ ]:
date_ranges = (
    test_df1
    .group_by('item_id')
    .agg([
        pl.col('timestamp').min().alias('earliest_date'),
        pl.col('timestamp').max().alias('latest_date')
    ])
)

**Fill 0 in range 2023-01-19 to 2025-02-08**

In [ ]:
import polars as pl
from datetime import date

# Step 1: Create a complete date range using a different approach
start_date = date(2025, 1, 19)
end_date = date(2025, 2, 8)
dates_df = pl.DataFrame({
    "timestamp": pl.date_range(start_date, end_date, "1d", eager=True)
})

# Step 2: Get unique item_ids
unique_item_ids = test_df1.get_column("item_id").unique()

# Step 3: Create DataFrame for the items
items_df = pl.DataFrame({"item_id": unique_item_ids})

# Create a cross product of dates and item_ids
complete_df = dates_df.join(items_df, how="cross")

# Step 4: Merge with original data to bring in existing values
result = complete_df.join(
    test_df1,
    on=["item_id", "timestamp"],
    how="left"
)

# Step 5: Fill missing target values with 0
result = result.with_columns(
    pl.col("OrderWeight_mean").fill_null(0)
)

# Sort by item_id and timestamp
result_test = result.sort(["item_id", "timestamp"])

result_test

**add holiday**

In [ ]:
result_test1 = result_test.join(
    holiday,
    left_on="timestamp",
    right_on="timestamp",
    how="inner"
)
result_test1

In [ ]:
pandas_df1 = result_test1.to_pandas()

pandas_df1 = pandas_df1.set_index(['item_id', 'timestamp'])
pandas_df1

In [ ]:
test_df2 = TimeSeriesDataFrame.from_data_frame(
    pandas_df1,
    id_column="item_id",
    timestamp_column="timestamp",
)
test_df2

**ข้ามไป tag predict** ก่อน

### tag resolve error

**Check duplicates index**

In [ ]:
# Check duplicates in the main time series dataframe
duplicates = ts_df.index.duplicated()
print("Duplicates in ts_df:", ts_df[duplicates])

# Do the same for known covariates, if used
if test_df2 is not None:
    duplicates_cov = test_df2.index.duplicated()
    print("Duplicates in test_df2:", test_df2[duplicates_cov])

In [ ]:
print(ts_df.index.names)         # ควรเป็น ['item_id', 'timestamp']
print(test_df2.index.names)      # ควรเป็น ['item_id', 'timestamp'] เช่นกัน

In [ ]:
ts_df = ts_df.sort_index()
test_df2 = test_df2.sort_index()

**if** *ValueError: cannot reindex on an axis with duplicate labels*

resolve them by aggregating them (e.g., by averaging values at the same (item_id, timestamp)) **

In [ ]:
test_df2_reset = test_df2.reset_index()
test_df2_grouped = test_df2_reset.groupby(['item_id', 'timestamp']).mean().reset_index()
test_df2_cleaned = test_df2_grouped.set_index(['item_id', 'timestamp']).sort_index()
test_df2_cleaned

### Predict

In [ ]:
predictions_df = predictor.predict(ts_df, known_covariates=test_df2)

ถ้าเกิด ValueError: cannot reindex on an axis with duplicate labels ไปที่ tag resolve

In [ ]:
import pandas as pd
y_pred = pd.DataFrame(predictions_df)
y_pred = y_pred.rename(columns={'mean': 'OrderWeight'})
# Convert TimeSeriesDataFrame to pandas DataFrame
predictions_pd = y_pred.to_pandas() if hasattr(y_pred, 'to_pandas') else y_pred
predictions_pd

In [ ]:
# If predictions_pd has a MultiIndex (common in TimeSeriesDataFrame), reset it
if isinstance(predictions_pd.index, pd.MultiIndex):
    predictions_pd = predictions_pd.reset_index()

# Convert submission to pandas if needed
submission_pd = ss.to_pandas() if hasattr(test_df, 'to_pandas') else ss

# Ensure both dataframes have proper date format
date_col_pred = 'timestamp' if 'timestamp' in predictions_pd.columns else 'OrderDate'

# Convert date columns to datetime if they're strings
if submission_pd['OrderDate'].dtype == 'object':
    submission_pd['OrderDate'] = pd.to_datetime(submission_pd['OrderDate'])

if predictions_pd[date_col_pred].dtype == 'object':
    predictions_pd[date_col_pred] = pd.to_datetime(predictions_pd[date_col_pred])


In [ ]:
# Create a mapping dataframe with just the needed columns
mapping_pd = predictions_pd[['item_id', date_col_pred, 'OrderWeight']]
if date_col_pred != 'date':
    mapping_pd = mapping_pd.rename(columns={date_col_pred: 'OrderDate'})
mapping_pd = mapping_pd.rename(columns={"item_id": 'key_cv_p'})
# mapping_pd = mapping_pd.rename(columns={"item_id": 'key_cv_p'})
mapping_pd

**merge predict with submission data**

In [ ]:
submission = pd.merge(
    submission_pd,
    mapping_pd,
    on=['key_cv_p', 'OrderDate'],
    how='left'
)
submission

**fill 0 data ที่ ไม่มีใน train**

In [ ]:
submission['OrderWeight'] = submission['OrderWeight'].fillna(0)

**จัดการ OrderWeight**

In [ ]:
## 1. แทนที่ค่าที่น้อยกว่า 0 เป็น 0
submission['OrderWeight'] = submission['OrderWeight'].apply(lambda x: 0 if x < 0 else x)

## 2. ปัดทศนิยมให้เหลือ 4 ตำแหน่ง
submission['OrderWeight'] = submission['OrderWeight'].round(4)

## ตรวจสอบผลลัพธ์
print("ค่าสถิติหลังปรับแต่ง:")
print(f"ประเภทข้อมูล: {submission['OrderWeight'].dtype}")
print(f"ค่าต่ำสุด: {submission['OrderWeight'].min()}")
print(f"ค่าสูงสุด: {submission['OrderWeight'].max()}")
print("\nตัวอย่างข้อมูล:")

In [ ]:
submission = submission.drop(columns = ['Unnamed: 0', 'key_cv_p'])
submission

**count zero**

In [ ]:
zero_count = (submission['OrderWeight'] == 0).sum()
print(f"จำนวนค่า 0 ในคอลัมน์ OrderWeight: {zero_count}")

In [ ]:
submission.to_csv('submission1.csv')